In [2]:
import pm4py
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from pm4py.algo.evaluation.earth_mover_distance import algorithm as earth_mover_distance

/home/lennc/.local/lib/python3.11/site-packages/pm4py/algo/evaluation/earth_mover_distance/__init__.py:21: UserWarning: The earth_mover_distance package will be removed in a future release.
  warnings.warn("The earth_mover_distance package will be removed in a future release.")


In [36]:
file_name = "PrepaidTravelCost"

In [38]:
case_id_key = "case:concept:name"

df = pm4py.read_xes(f'../logs/{file_name}.xes')
# df = pd.read_csv(f'../logs/{file_name}.{attribute_key}.csv')
# df["time:timestamp"] = pd.to_datetime(df['time:timestamp'])
# df["case:concept:name"] = df['case:concept:name'].astype(str)
# df["concept:name"] = df['concept:name'].astype(str)

# df = df.dropna(subset=[case_attribute_key]).reset_index() # filter empty rows
df.drop(df[df["lifecycle:transition"] == "start"].index, inplace=True)

parsing log, completed traces :: 100%|██████████| 2099/2099 [00:00<00:00, 5077.43it/s]


In [39]:
df.head()

,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Permit travel permit number,case:Task,case:OrganizationalEntity,case:RequestedAmount,...,case:Permit BudgetNumber,case:Permit ProjectNumber,case:Project,case:concept:name,case:Permit OrganizationalEntity,case:Permit RequestedBudget,case:Cost Type,case:Permit id,case:Permit ActivityNumber,case:RfpNumber
0,st_step 73555_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-01-09 13:48:43+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,budget 6198,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551
1,st_step 73554_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-01-09 13:48:55+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,budget 6198,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551
2,st_step 73558_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-12 10:40:27+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,budget 6198,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551
3,st_step 73559_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-12 10:41:59+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,budget 6198,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551
4,st_step 73557_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-12 10:53:07+00:00,MISSING,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,budget 6198,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551


In [40]:
pm4py.get_trace_attributes(df)

['case:Rfp_id',
 'case:Permit travel permit number',
 'case:Task',
 'case:OrganizationalEntity',
 'case:RequestedAmount',
 'case:Activity',
 'case:Permit TaskNumber',
 'case:Permit BudgetNumber',
 'case:Permit ProjectNumber',
 'case:Project',
 'case:concept:name',
 'case:Permit OrganizationalEntity',
 'case:Permit RequestedBudget',
 'case:Cost Type',
 'case:Permit id',
 'case:Permit ActivityNumber',
 'case:RfpNumber']

In [41]:
sum(pm4py.get_variants(df).values())

2099

In [42]:
len(pm4py.get_variants(df).values())

202

In [11]:
df = pd.read_csv("../logs/Hospital_log.Age:1.csv")
df = df.groupby(["case:concept:name"]).first()
display(df[df['case:Age:1'].isna()])
display(df[df['case:Age:1'].notna()])
print(f"no age:1: {df[df['case:Age:1'].isna()]}")
print(f"no age:1: {df[df['case:Age:1'].notna()]}")

no age:1:                    Unnamed: 0                             concept:name  \
case:concept:name                                                        
0                           0                  1e consult poliklinisch   
6                         605  verlosk.-gynaec. korte kaart kosten-out   
9                        1195  verlosk.-gynaec.   jaarkaart kosten-out   
10                       1203  verlosk.-gynaec. korte kaart kosten-out   
11                       1464  verlosk.-gynaec. korte kaart kosten-out   
...                       ...                                      ...   
1138                   150151          echografie  - genitalia interna   
1139                   150182                         coupe ter inzage   
1140                   150208            aanname laboratoriumonderzoek   
1141                   150235        e.c.g.      - elektrocardiografie   
1142                   150265            aanname laboratoriumonderzoek   

                           